# Companies House Public Data API Project Using PySpark

## 1. Library Imports

In [9]:
# for making HTTP requests to the Companies House API or any web service
import requests
# for parsing JSON responses like API outputs or serialising PYthon objects into JSON
import json
# for introducing pauses between API calls to respect rate limits and avoid being blocked
import time
# for writing tabular data to CSV files (e.g., saving scraped or API data)
import csv
# for interacting with the operating system, such as checking/creating directories, working with environment variables, or building file paths
import os
# pandas for loading and manipulating tabular data
import pandas as pd
# selenium is used for browser automation to load and scrape websites dynamically
from selenium import webdriver
# allows specifying options for Chrome, like setting window size or running in headless mode
from selenium.webdriver.chrome.options import Options
# provides a way to locate elements on the web page (by ID, class, XPath, etc.)
from selenium.webdriver.common.by import By
# enables waiting for certain conditions to occur (e.g., an element becoming clickable)
from selenium.webdriver.support.ui import WebDriverWait
# provides built-in expected conditions like visibility, clickability, or presence of elements
from selenium.webdriver.support import expected_conditions as EC
# lets one manage the ChromeDriver service, such as launching it or stopping it cleanly
from selenium.webdriver.chrome.service import Service

## 2. Configuration Loading

In [10]:
def load_api_key(filepath="config.txt"):
    """
    Load an API Key from a config file.
    The file must contain a line likeL API_KEY=your-key-here
    """
    with open(filepath, "r") as file:
        for line in file:
            if line.startswith("API_KEY="):
                return line.strip().split("=")[1]
    raise ValueError("API_KEY not found in config file.")

In [11]:
# load API key from config file - this avoids hardcoding the key into the script
API_KEY = load_api_key()
# set the base URL for all API calls
BASE_URL = "https://api.company-information.service.gov.uk"
# specify that the client expects JSON responses from the API
HEADERS = {"Accept": "application/json"}
# file name where all fetched company profile data will be saved as a CSV
OUTPUT_FILE = "company_profiles.csv"
# time delay (in seconds) between consecutive API requests to avoid hitting rate limits
DELAY_BETWEEN_REQUESTS = 1

## 3. Scraping FTSE 100 Constituents

### 3.1. Scraping Basic Company Information

In [12]:
# --- Setup ---

# URL of the FTSE 100 constituent table
url = "https://www.londonstockexchange.com/indices/ftse-100/constituents/table"

# set up Chrome options
options = Options()
# ensure full content loads properly
options.add_argument("--window-size=1920,1080")

# launch the Chrome browser with the specified options
driver = webdriver.Chrome(service=Service(), options=options)

# navigate to the URL
driver.get(url)

# set up an explicit wait of 15 seconds to be used throughout the script
wait = WebDriverWait(driver, 15)

# --- Accept Cookie Consent ---

try:
    # wait for the "Accept all cookies" button to be clickable and then click it
    cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Accept all cookies']")))
    cookie_btn.click()
    print("[INFO] Accepted cookie consent.")
except Exception as e:
    # if the button doesn't appear or isn't clickable, continue without crashing
    print(f"[WARN] Cookie button not found: {e}")

# --- Scrape All 5 Pages ---

# initialise a list to store tuples
data = []

# loop through pages 1 to 5
for page_num in range(1, 6):
    print(f"[INFO] Scraping page {page_num}...")

    try:
        # wait until the table rows (one per company) are loaded
        wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "tr.medium-font-weight")
        ))
        time.sleep(1)  # slight pause to ensure rows are populated
        rows = driver.find_elements(By.CSS_SELECTOR, "tr.medium-font-weight")
        print(f"[INFO] Found {len(rows)} rows.")
    except Exception as e:
        print(f"[ERROR] Table not found on page {page_num}: {e}")
        continue

    # extract ticker and company name from each row
    for row in rows:
        try:
            ticker = row.find_element(By.CSS_SELECTOR, "td.instrument-tidm a").text.strip()
            name = row.find_element(By.CSS_SELECTOR, "td.instrument-name a").text.strip()
            currency = row.find_element(By.CSS_SELECTOR, "td.instrument-currency").text.strip()
            market_cap = row.find_element(By.CSS_SELECTOR, "td.instrument-marketcapitalization").text.strip()
            price = row.find_element(By.CSS_SELECTOR, "td.instrument-lastprice").text.strip()
            change = row.find_element(By.CSS_SELECTOR, "td.instrument-netchange").text.strip()
            change_pct = row.find_element(By.CSS_SELECTOR, "td.instrument-percentualchange").text.strip()

            data.append((ticker, name, currency, market_cap, price, change, change_pct))
        except Exception as e:
            print(f"[WARN] Failed to parse a row: {e}")

    # click the next page link unless we're on the last page
    if page_num < 5:
        try:
            # wait for the link to the next page number to be clickable
            next_button = wait.until(EC.element_to_be_clickable(
                (By.XPATH, f"//a[@class='page-number' and text()='{page_num + 1}']")
            ))
            # scroll to it as some buttons won't click if off-screen
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.5) # small delay before clicking
            next_button.click()
            time.sleep(2.5) # allow the new page to load
        except Exception as e:
            print(f"[ERROR] Could not navigate to page {page_num + 1}: {e}")
            break

# close the browser after scraping
driver.quit()

# --- Save to CSV ---

# write the scraped data to a CSV file
with open("ftse100_lse.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Ticker", "Company Name", "Currency", "Market Cap (m)", "Price", "Change", "Change %"])
    writer.writerows(data)

print(f"\nRetrieved {len(data)} companies from LSE FTSE 100.")

[INFO] Accepted cookie consent.
[INFO] Scraping page 1...
[INFO] Found 20 rows.
[INFO] Scraping page 2...
[INFO] Found 20 rows.
[INFO] Scraping page 3...
[INFO] Found 20 rows.
[INFO] Scraping page 4...
[INFO] Found 20 rows.
[INFO] Scraping page 5...
[INFO] Found 20 rows.

Retrieved 100 companies from LSE FTSE 100.


### 3.2. Scraping Financial Statements

In [14]:
# --- Helper function to extract and save financial tables ---
def extract_and_save_table(driver, section_title, folder_path, filename):
    try:
        # wait for the table that contains the specified section title to appear in the DOM
        section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//th[contains(text(), '{section_title}')]/ancestor::table"))
        )

        # find all rows in the located table
        rows = section.find_elements(By.CSS_SELECTOR, "tr")
        table_data = []

        # loop over each row in the table
        for row in rows:
            # extract both header and data cells in the row
            cols = row.find_elements(By.CSS_SELECTOR, "th, td")
            row_data = []
            
            for col in cols:
                # remove any nested child elements like tooltips (e.g. "What's this?") to get clean clean text
                driver.execute_script("""
                    const el = arguments[0];
                    el.querySelectorAll('app-whats-this, .whats-this, .tooltip').forEach(n => n.remove());
                """, col)
                # clean the text by stripping whitespace and removing known tooltip phrases
                clean_text = col.text.strip().replace("What's this?", "").strip()
                row_data.append(clean_text)

            # append the cleaned row to the main data list
            table_data.append(row_data)

        # write the final cleaned table to a CSV file
        with open(os.path.join(folder_path, filename), "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerows(table_data)

        print(f"  [INFO] Saved {filename}")
    except Exception as e:
        print(f"  [WARN] Could not find or save {filename}: {e}")

# --- Setup ---
base_url = "https://www.londonstockexchange.com"
constituents_url = f"{base_url}/indices/ftse-100/constituents/table"

# --- Configure browser settings ---
options = Options()
options.add_argument("--window-size=1920,1080") # set full HD resolution to enure page loads correctly

# launch chrome with the specified options
driver = webdriver.Chrome(service=Service(), options=options)
driver.get(constituents_url)
wait = WebDriverWait(driver, 15)

# --- Accept Cookies ---
try:
    # wait for and click the cookie consent button
    cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Accept all cookies']")))
    cookie_btn.click()
    print("[INFO] Accepted cookie consent.")
except Exception as e:
    # skip if cookie consent is not found
    print(f"[WARN] Cookie button not found: {e}")

# --- Extract tickers and links ---
ticker_links = [] # list to store (ticker, full URL) tuples

# loop through all 5 pages of FTSE100 constituents
for page_num in range(1, 6):
    print(f"[INFO] Reading page {page_num}...")

    # ensure all rows are loaded before proceeding
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "tr.medium-font-weight")))
    time.sleep(1)
    rows = driver.find_elements(By.CSS_SELECTOR, "tr.medium-font-weight")

    for row in rows:
        try:
            # extract ticker symbol and its company page URL
            ticker_el = row.find_element(By.CSS_SELECTOR, "td.instrument-tidm a")
            ticker = ticker_el.text.strip()
            relative_url = ticker_el.get_attribute("href")
            full_url = base_url + relative_url if relative_url.startswith("/") else relative_url
            ticker_links.append((ticker, full_url))
        except Exception as e:
            print(f"[WARN] Could not parse row: {e}")

    # navigate to the next page unless already on the last one
    if page_num < 5:
        try:
            next_btn = wait.until(EC.element_to_be_clickable(
                (By.XPATH, f"//a[@class='page-number' and text()='{page_num + 1}']")
            ))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
            time.sleep(0.5)
            next_btn.click()
            time.sleep(2.5)
        except Exception as e:
            print(f"[ERROR] Could not go to page {page_num + 1}: {e}")
            break

# --- Visit Each Ticker and Scrape Financials ---
os.makedirs("financial_statements", exist_ok=True)

for ticker, link in ticker_links:
    print(f"[INFO] Scraping financials for {ticker}...")

    try:
        driver.get(link)
        print("  [STEP] Navigated to ticker page.")

        # handle cookie banner again on company page if needed
        try:
            cookie_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Accept all cookies']"))
            )
            cookie_btn.click()
            print(f"  [INFO] Accepted cookie consent on {ticker} page.")
        except:
            print("  [STEP] No cookie banner on ticker page.")

        # click the "Fundamentals" tab with scroll
        fundamentals_tab = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Fundamentals")))
        driver.execute_script("arguments[0].scrollIntoView(true);", fundamentals_tab)
        time.sleep(0.5)
        fundamentals_tab.click()
        print("  [STEP] Clicked 'Fundamentals' tab.")
        time.sleep(2)

        # expand all accordion sections (+ buttonns) for financial tables
        print("  [STEP] Expanding all financial sections...")
        expand_buttons = driver.find_elements(By.CSS_SELECTOR, "th.hide-on-desktop .accordion-toggler")
        
        if not expand_buttons:
            print("  [WARN] No expand buttons found.")
        else:
            for i, btn in enumerate(expand_buttons, 1):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", btn)
                    time.sleep(0.2)
                    driver.execute_script("arguments[0].click();", btn)
                    print(f"  [INFO] Expanded section #{i}")
                    time.sleep(1)
                except Exception as e:
                    print(f"  [WARN] Failed to expand section #{i}: {e}")

        # create a directory for this ticker
        company_folder = os.path.join("financial_statements", ticker)
        os.makedirs(company_folder, exist_ok=True)

        # save the three primary financial tables to CSV
        extract_and_save_table(driver, "Income statement", company_folder, "income_statement.csv")
        extract_and_save_table(driver, "Balance sheet", company_folder, "balance_sheet.csv")
        extract_and_save_table(driver, "Ratios", company_folder, "ratios.csv")

    except Exception as e:
        print(f"[ERROR] Failed to scrape {ticker}: {e}")
        continue

# --- Cleanup ---
driver.quit()
print("\nCompleted scraping financial statements.")

[INFO] Accepted cookie consent.
[INFO] Reading page 1...
[INFO] Reading page 2...
[INFO] Reading page 3...
[INFO] Reading page 4...
[INFO] Reading page 5...
[INFO] Scraping financials for ENT...
  [STEP] Navigated to ticker page.
  [STEP] No cookie banner on ticker page.
  [STEP] Clicked 'Fundamentals' tab.
  [STEP] Expanding all financial sections...
  [INFO] Expanded section #1
  [INFO] Expanded section #2
  [INFO] Expanded section #3
  [INFO] Saved income_statement.csv
  [INFO] Saved balance_sheet.csv
  [INFO] Saved ratios.csv
[INFO] Scraping financials for AAF...
  [STEP] Navigated to ticker page.
  [STEP] No cookie banner on ticker page.
  [STEP] Clicked 'Fundamentals' tab.
  [STEP] Expanding all financial sections...
  [INFO] Expanded section #1
  [INFO] Expanded section #2
  [INFO] Expanded section #3
  [INFO] Saved income_statement.csv
  [INFO] Saved balance_sheet.csv
  [INFO] Saved ratios.csv
[INFO] Scraping financials for MNDI...
  [STEP] Navigated to ticker page.
  [STEP] No

## 4. Companies House API Integration

In [17]:
# fetches the core company profile from the Companies House API
def fetch_company_profile(company_number):
    # construct the API URL for the company's main profile
    url = f"{BASE_URL}/company/{company_number}"
    try:
        # send GET request with basic auth (API key only)
        response = requests.get(url, auth=(API_KEY, ""), headers=HEADERS)
        if response.status_code == 200:
            # return JSON data if successful
            return response.json()
        else:
            # log status code if note successful
            print(f"[{response.status_code}] Failed to fetch {company_number}")
            return None
    except Exception as e:
        # catch and log connection or parsing errors
        print(f"[ERROR] {company_number}: {str(e)}")
        return None

# retrieves a list of up to 3 officer (director) names for the company
def fetch_officers(company_number):
    url = f"{BASE_URL}/company/{company_number}/officers"
    response = requests.get(url, auth=(API_KEY, ""), headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # limit to first 3 officers
        officers = data.get("items", [])[:3]
        # return only officer names, filtering out any nulls
        return [str(o.get("name", "Unknown")) for o in officers if o.get("name")]
    # return empty list if request fails or no officers
    return []

# retrieves up to 5 most recent filing history entries
def fetch_filing_history(company_number):
    url = f"{BASE_URL}/company/{company_number}/filing-history"
    response = requests.get(url, auth=(API_KEY, ""), headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # limit to 5 filings
        filings = data.get("items", [])[:5]
        # format: "filing_type on date"
        return [f"{str(f.get('type', 'UNKNOWN'))} on {str(f.get('date', 'UNKNOWN'))}" for f in filings]
    return []

# retrieves up to 3 charge entries (e.g. mortgage or secured lending)
def fetch_charges(company_number):
    url = f"{BASE_URL}/company/{company_number}/charges"
    response = requests.get(url, auth=(API_KEY, ""), headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # limits to 3 charges
        charges = data.get("items", [])[:3]
        # get charge codes, safely convert to string
        return [str(c.get("charge_code", "Unknown")) for c in charges if c.get("charge_code")]
    return []

# retrieves up to 3 names of Persons with Significant Control (PSC)
def fetch_psc(company_number):
    url = f"{BASE_URL}/company/{company_number}/persons-with-significant-control"
    response = requests.get(url, auth=(API_KEY, ""), headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # limit to 3 PSC entries
        pscs = data.get("items", [])[:3]
        # return PSC names if present
        return [str(p.get("name", "Unknown")) for p in pscs if p.get("name")]
    return []

In [2]:
# need to translate company names into company numbers on Companies House - how?
# need to integrate stock price history data - Yahoo Finance API?

In [16]:
def main():
    # print the number of companies to be processed
    print(f"Fetching data for {len(COMPANY_NUMBERS)} companies...")

    # initialise an empty list to collect all results (one dict per company)
    results = []

    # loop over each company number in the list
    for number in COMPANY_NUMBERS:
        # fetch the company's profile (core metadata)
        data = fetch_company_profile(number)

        # if data was successfully retrieved, enrich and store it
        if data:
            results.append({
                # extract basic company information from the profile
                "company_number": data.get("company_number"),
                "company_name": data.get("company_name"),
                "status": data.get("company_status"),
                "date_of_creation": data.get("date_of_creation"),
                "type": data.get("type"),
                "company_category": data.get("company_category"),
                "jurisdiction": data.get("jurisdiction"),

                # financial reporting data
                "last_accounts_date": data.get("accounts", {}).get("last_accounts", {}).get("made_up_to"),
                "next_accounts_due": data.get("accounts", {}).get("next_due"),

                # latest confirmation statement date
                "confirmation_statement_date": data.get("confirmation_statement", {}).get("last_made_up_to"),

                # whether the company has a history of insolvency
                "has_insolvency_history": data.get("has_insolvency_history"),

                # flatten the address JSON structure into a string
                "registered_office_address": json.dumps(data.get("registered_office_address", {})),

                # list of SIC (Standard Industrial Classification) codes, joined into one string
                "sic_codes": ",".join(data.get("sic_codes", [])),

                # enriched data from additional endpoints
                "officers": "; ".join(fetch_officers(number)), # top 3 officers
                "recent_filings": "; ".join(fetch_filing_history(number)), # latest 5 filings
                "charges": "; ".join(fetch_charges(number)), # top 3 charge codes
                "psc": "; ".join(fetch_psc(number)) # up to 3 persons with significant control
            })

        # wait between requests to avoid hitting rate limits
        time.sleep(DELAY_BETWEEN_REQUESTS)

        # save the current state of the results to CSV after each company
        keys = results[0].keys() if results else [] # get column headers from first result
        with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(results)

        # log the progress
        print(f"Saved {len(results)} company profiles to {OUTPUT_FILE}")

# entry point of the script - only runs when file is executed directly
if __name__ == "__main__":
    main()

NameError: name 'COMPANY_NUMBERS' is not defined